In [9]:
# Core Packages
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter.scrolledtext import *
import tkinter.filedialog

#Other Packages
import re
import PyPDF2
from docx import Document
import time
timestr = time.strftime("%Y%m%d-%H%M%S")

# Web Scraping Pkg 
from bs4 import BeautifulSoup
from urllib.request import urlopen

# NLP Pkgs
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import heapq 
import re


#Summary Function
def nltk_summarizer(raw_text):
    
    # Removing Square Brackets and Extra Spaces
    raw_text = re.sub(r'\[[0-9]*\]', ' ', raw_text)
    raw_text = re.sub(r'\s+', ' ', raw_text)

    # Removing special characters and digits
    formatted_raw_text = re.sub('[^a-zA-Z]', ' ', raw_text)
    formatted_raw_text = re.sub(r'\s+', ' ', formatted_raw_text)

    word_list = nltk.word_tokenize(formatted_raw_text)
    
    stopWords = set(stopwords.words("english"))

    word_frequencies = {}  
    for word in word_list:  
        if word not in stopWords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
        
        
    sentence_list = nltk.sent_tokenize(raw_text)
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]



    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences) 
    
    return summary


 # Structure and Layout
window = Tk()
window.title("Summaryzer GUI")
window.geometry("700x400")
window.config(background='black')

style = ttk.Style(window)
style.configure('lefttab.TNotebook', tabposition='wn',)


# TAB LAYOUT
tab_control = ttk.Notebook(window,style='centertab.TNotebook')
 
tab1 = ttk.Frame(tab_control)
tab2 = ttk.Frame(tab_control)
tab3 = ttk.Frame(tab_control)
tab4 = ttk.Frame(tab_control)
tab5 = ttk.Frame(tab_control)
tab6 = ttk.Frame(tab_control)


# ADD TABS TO NOTEBOOK
tab_control.add(tab1, text=f'{"Home":^20s}')
tab_control.add(tab2, text=f'{"Text File":^20s}')
tab_control.add(tab3, text=f'{"PDF":^20s}')
tab_control.add(tab4, text=f'{"Word Document":^20s}')
tab_control.add(tab5, text=f'{"URL":^20s}')
tab_control.add(tab6, text=f'{"About ":^20s}')

label1 = Label(tab1, text= 'Summarizer',padx=5, pady=5)
label1.grid(column=0, row=0)
 
label2 = Label(tab2, text= 'Text Summary',padx=5, pady=5)
label2.grid(column=1, row=0)
                
label3 = Label(tab3, text= 'PDF Summary',padx=5, pady=5)
label3.grid(column=1, row=0)
 
label4 = Label(tab4, text= 'Word Summary',padx=5, pady=5)
label4.grid(column=1, row=0)

label5 = Label(tab5, text= 'URL',padx=5, pady=5)
label5.grid(column=1, row=0)

label6 = Label(tab6, text= 'About',padx=5, pady=5)
label6.grid(column=1, row=0)


tab_control.pack(expand=1, fill='both')

####################################################################################################################                
#Home Tab
# Functions 
def get_summary():
    raw_text = str(entry.get('1.0',tk.END))
    final_text =nltk_summarizer(raw_text)
    result = '\nSummary:{}'.format(final_text)
    tab1_display.insert(tk.END,result)

def save_summary():
    raw_text = str(entry.get('1.0',tk.END))
    final_text = nltk_summarizer(raw_text)            
    file_name = "yoursummary" + timestr + '.txt'
    with open(file_name,'w') as f:
         f.write(final_text)
    result ='\nName of File: {} , \nSummary:{}'.format(file_name,final_text)
    tab1_display.insert(tk.END,result)
           
                
# Clear entry widget
def clear_text():
    entry.delete('1.0',END)

def clear_display_result():
    tab1_display.delete('1.0',END)


# Clear Text  with position 1.0
def clear_text_file():
    displayed_file.delete('1.0',END)

 
                
l1=Label(tab1,text="Enter Text To Summarize")
l1.grid(row=1,column=0)

entry=ScrolledText(tab1,height=7)
entry.grid(row=2,column=0,columnspan=2,padx=5,pady=5)

# BUTTONS
button1=Button(tab1,text="Reset",command=clear_text, width=12,bg='#03A9F4',fg='#fff')
button1.grid(row=4,column=0,padx=10,pady=10)

button2=Button(tab1,text="Summarize",command=get_summary, width=12,bg='#03A9F4',fg='#fff')
button2.grid(row=4,column=1,padx=10,pady=10)

button3=Button(tab1,text="Clear Result", command=clear_display_result,width=12,bg='#03A9F4',fg='#fff')
button3.grid(row=5,column=0,padx=10,pady=10)

button4=Button(tab1,text="Save",command=save_summary ,width=12,bg='#03A9F4',fg='#fff')
button4.grid(row=5,column=1,padx=10,pady=10)

# Display Screen For Result
tab1_display = ScrolledText(tab1,height=10)
tab1_display.grid(row=7,column=0, columnspan=3,padx=5,pady=5)

###################################################################################################################### 
#Text File Processing Tab

#Functions

# Open File to Read and Process
def openfile_text():
    file_txt = tkinter.filedialog.askopenfilename(filetypes=(("Text Files",".txt"),("All files","*")))
    read_text = open(file_txt).read()
    displayed_text.insert(tk.END,read_text)


def get_text_summary():
    raw_text = displayed_text.get('1.0',tk.END)
    final_text = nltk_summarizer(raw_text)
    result = '\nSummary:{}'.format(final_text)
    tab2_display_text.insert(tk.END,result)
                
def save_text_summary():
    raw_text = displayed_text.get('1.0',tk.END)
    final_text = nltk_summarizer(raw_text)            
    file_name = "text_file_summary" + timestr + '.txt'
    with open(file_name,'w') as f:
         f.write(final_text)
    result ='\nName of File: {} , \nSummary:{}'.format(file_name,final_text)
    tab2_display_text.insert(tk.END,result)
        


# Clear Text  with position 1.0
def clear_text_file():
    displayed_text.delete('1.0',END)

# Clear Result of Functions
def clear_text_result():
    tab2_display_text.delete('1.0',END)

                
l2=Label(tab2,text="Open File To Summarize")
l2.grid(row=1,column=1)

displayed_text = ScrolledText(tab2,height=7)
displayed_text.grid(row=2,column=0, columnspan=3,padx=5,pady=3)

# BUTTONS FOR SECOND TAB/FILE READING TAB
b0=Button(tab2,text="Open File", width=12,command=openfile_text,bg='#03A9F4',fg='#fff')
b0.grid(row=3,column=0,padx=10,pady=10)

b1=Button(tab2,text="Reset ", width=12,command=clear_text_file,bg="#03A9F4",fg='#fff')
b1.grid(row=3,column=1,padx=10,pady=10)

b2=Button(tab2,text="Summarize", width=12,command=get_text_summary,bg='#03A9F4',fg='#fff')
b2.grid(row=3,column=2,padx=10,pady=10)

b3=Button(tab2,text="Clear Result", width=12,command=clear_text_result,bg='#03A9F4',fg='#fff')
b3.grid(row=5,column=0,padx=10,pady=10)

b4=Button(tab2,text="Close", width=12,command=window.destroy,bg='#03A9F4',fg='#fff')
b4.grid(row=5,column=2,padx=10,pady=10)
                
b5=Button(tab2,text="Save",command=save_text_summary,width=12,bg='#03A9F4',fg='#fff')
b5.grid(row=5,column=1,padx=10,pady=10)


# Display Screen
tab2_display_text = ScrolledText(tab2,height=10)
tab2_display_text.grid(row=7,column=0, columnspan=3,padx=5,pady=5)

# Allows you to edit
tab2_display_text.config(state=NORMAL)
         
                
######################################################################################################################                
#PDF FILE PROCESSING TAB
# Open File to Read and Process
def openfile_pdf():
    file_pdf = tkinter.filedialog.askopenfilename(filetypes=(("PDFs",".pdf"),("All files","*")))
    pdfFileObject = open(file_pdf , 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
    count = pdfReader.numPages
    for i in range(count):
        page = pdfReader.getPage(i)
        pdf_data=page.extractText()
        displayed_pdf.insert(tk.END,pdf_data)


def get_pdf_summary():
    raw_text = displayed_pdf.get('1.0',tk.END)
    final_text = nltk_summarizer(raw_text)
    result = '\nSummary:{}'.format(final_text)
    tab3_display_pdf.insert(tk.END,result)
                
def save_pdf_summary():
    raw_text = displayed_pdf.get('1.0',tk.END)
    final_text = nltk_summarizer(raw_text)            
    file_name = "pdf_file_summary" + timestr + '.txt'
    with open(file_name,'w') as f:
         f.write(final_text)
    result ='\nName of File: {} , \nSummary:{}'.format(file_name,final_text)
    tab3_display_pdf.insert(tk.END,result)
       


# Clear Text  with position 1.0
def clear_pdf_file():
    displayed_pdf.delete('1.0',END)

# Clear Result of Functions
def clear_pdf_result():
    tab3_display_pdf.delete('1.0',END)



l3=Label(tab3,text="Open PDF File To Summarize")
l3.grid(row=1,column=1)

displayed_pdf = ScrolledText(tab3,height=7)
displayed_pdf.grid(row=2,column=0, columnspan=3,padx=5,pady=3)

# BUTTONS FOR SECOND PDF READING TAB
b0=Button(tab3,text="Open File", width=12,command=openfile_pdf,bg='#03A9F4',fg='#fff')
b0.grid(row=3,column=0,padx=10,pady=10)

b1=Button(tab3,text="Reset ", width=12,command=clear_pdf_file,bg="#03A9F4",fg='#fff')
b1.grid(row=3,column=1,padx=10,pady=10)

b2=Button(tab3,text="Summarize", width=12,command=get_pdf_summary,bg='#03A9F4',fg='#fff')
b2.grid(row=3,column=2,padx=10,pady=10)

b3=Button(tab3,text="Clear Result", width=12,command=clear_pdf_result,bg='#03A9F4',fg='#fff')
b3.grid(row=5,column=0,padx=10,pady=10)

b4=Button(tab3,text="Close", width=12,command=window.destroy,bg='#03A9F4',fg='#fff')
b4.grid(row=5,column=2,padx=10,pady=10)
                
b5=Button(tab3,text="Save",command=save_pdf_summary,width=12,bg='#03A9F4',fg='#fff')
b5.grid(row=5,column=1,padx=10,pady=10)


# Display Screen
# tab2_display_text = Text(tab2)
tab3_display_pdf = ScrolledText(tab3,height=10)
tab3_display_pdf.grid(row=7,column=0, columnspan=3,padx=5,pady=5)

# Allows you to edit
tab3_display_pdf.config(state=NORMAL)
                
#########################################################################################################################
#Word Doc PROCESSING TAB
# Open File to Read and Process
def openfile_wrd():
    file_doc = tkinter.filedialog.askopenfilename(filetypes=(("Word",".docx"),("All files","*")))
    document = Document(file_doc)
    for para in document.paragraphs:
        word_doc=para.text
        displayed_wrd.insert(tk.END,word_doc)


def get_wrd_summary():
    raw_text = displayed_wrd.get('1.0',tk.END)
    final_text = nltk_summarizer(raw_text)
    result = '\nSummary:{}'.format(final_text)
    tab4_display_wrd.insert(tk.END,result)
                
def save_wrd_summary():
    raw_text = displayed_wrd.get('1.0',tk.END)
    final_text = nltk_summarizer(raw_text)            
    file_name = "word_file_summary" + timestr + '.txt'
    with open(file_name,'w') as f:
         f.write(final_text)
    result ='\nName of File: {} , \nSummary:{}'.format(file_name,final_text)
    tab4_display_wrd.insert(tk.END,result)
       


# Clear Text  with position 1.0
def clear_wrd_file():
    displayed_wrd.delete('1.0',END)

# Clear Result of Functions
def clear_wrd_result():
    tab4_display_wrd.delete('1.0',END)



l4=Label(tab4,text="Open Word File To Summarize")
l4.grid(row=1,column=1)

displayed_wrd = ScrolledText(tab4,height=7)
displayed_wrd.grid(row=2,column=0, columnspan=3,padx=5,pady=3)

# BUTTONS FOR SECOND Word Doc READING TAB
b0=Button(tab4,text="Open File", width=12,command=openfile_wrd,bg='#03A9F4',fg='#fff')
b0.grid(row=3,column=0,padx=10,pady=10)

b1=Button(tab4,text="Reset ", width=12,command=clear_wrd_file,bg="#03A9F4",fg='#fff')
b1.grid(row=3,column=1,padx=10,pady=10)

b2=Button(tab4,text="Summarize", width=12,command=get_wrd_summary,bg='#03A9F4',fg='#fff')
b2.grid(row=3,column=2,padx=10,pady=10)

b3=Button(tab4,text="Clear Result", width=12,command=clear_wrd_result,bg='#03A9F4',fg='#fff')
b3.grid(row=5,column=0,padx=10,pady=10)

b4=Button(tab4,text="Close", width=12,command=window.destroy,bg='#03A9F4',fg='#fff')
b4.grid(row=5,column=2,padx=10,pady=10)
                
b5=Button(tab4,text="Save",command=save_wrd_summary,width=12,bg='#03A9F4',fg='#fff')
b5.grid(row=5,column=1,padx=10,pady=10)


# Display Screen
# tab2_display_text = Text(tab2)
tab4_display_wrd = ScrolledText(tab4,height=10)
tab4_display_wrd.grid(row=7,column=0, columnspan=3,padx=5,pady=5)

# Allows you to edit
tab4_display_wrd.config(state=NORMAL)                            
                
                
#########################################################################################################################                
                
# URL TAB

# Clear For URL
def clear_url_entry():
    url_entry.delete(0,END)

def clear_url_display():
    tab5_display_text.delete('1.0',END)
                
# Clear Text  with position 1.0
def clear_display():
    url_display.delete('1.0',END)


# Fetch Text From Url
def get_text():
    raw_text = str(url_entry.get())           
    page = urlopen(raw_text)
    soup = BeautifulSoup(page)            
    fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))          
    url_display.insert(tk.END,fetched_text)

def get_url_summary():
    raw_text = url_display.get('1.0',tk.END)           
    final_text = nltk_summarizer(raw_text)
    result = '\nSummary:{}'.format(final_text)
    tab5_display_text.insert(tk.END,result)
                
def save_url_summary():
    raw_text = url_display.get('1.0',tk.END)
    final_text = nltk_summarizer(raw_text)
    #print(final_text)
    file_name = "url_summary" + timestr + '.txt'
    with open(file_name,'w') as f:
         f.write(final_text)
    result ='\nName of File: {} , \nSummary:{}'.format(file_name,final_text)
    tab5_display_text.insert(tk.END,result)
       
                
l5=Label(tab5,text="Enter URL To Summarize")
l5.grid(row=1,column=0)

raw_entry=StringVar()
url_entry=Entry(tab5,textvariable=raw_entry,width=50)
url_entry.grid(row=1,column=1)
                

# BUTTONS
b0=Button(tab5,text="Reset URL",command=clear_url_entry, width=12,bg='#03A9F4',fg='#fff')
b0.grid(row=4,column=0,padx=10,pady=10)

b1=Button(tab5,text="Get Text",command=get_text, width=12,bg='#03A9F4',fg='#fff')
b1.grid(row=4,column=1,padx=10,pady=10)

b2=Button(tab5,text="Clear Result", command=clear_url_display,width=12,bg='#03A9F4',fg='#fff')
b2.grid(row=5,column=1,padx=10,pady=10)

b3=Button(tab5,text="Summarize",command=get_url_summary, width=12,bg='#03A9F4',fg='#fff')
b3.grid(row=4,column=2,padx=10,pady=10)
                
b4=Button(tab5,text="Reset Display", width=12,command=clear_display,bg='#03A9F4',fg='#fff')
b4.grid(row=5,column=0,padx=10,pady=10)
                
b5=Button(tab5,text="Save",command=save_url_summary,width=12,bg='#03A9F4',fg='#fff')
b5.grid(row=5,column=2,padx=10,pady=10)


# Display Screen For Result
url_display = ScrolledText(tab5,height=10)
url_display.grid(row=7,column=0, columnspan=3,padx=5,pady=5)


tab5_display_text = ScrolledText(tab5,height=10)
tab5_display_text.grid(row=10,column=0, columnspan=3,padx=5,pady=5)


# About TAB
about_label1 = Label(tab6,text="Automatic Text Summarizer",pady=5,padx=5)
about_label1.grid(column=1,row=1)

about_label2 = Label(tab6,text="Containing Text Summary for  \n Text File,PDF,Word Document & Website Text",pady=5,padx=5)
about_label2.grid(column=1,row=2)
 
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ROHIT\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-9-2f67c3dadd9b>", line 260, in openfile_pdf
    pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
  File "C:\Users\ROHIT\Anaconda3\lib\site-packages\PyPDF2\pdf.py", line 1084, in __init__
    self.read(stream)
  File "C:\Users\ROHIT\Anaconda3\lib\site-packages\PyPDF2\pdf.py", line 1901, in read
    raise utils.PdfReadError("Could not find xref table at specified location")
PyPDF2.utils.PdfReadError: Could not find xref table at specified location
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ROHIT\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-9-2f67c3dadd9b>", line 259, in openfile_pdf
    pdfFileObject = open(file_pdf , 'rb')
FileNotFoundError: [Errno 2] No such file or directory: ''
